<a href="https://colab.research.google.com/github/dhruvbhavsar0612/custom-sentiment-analyzer/blob/master/research/llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf
!pip install -q transformers einops accelerate langchain bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [2]:
# word embeddings

!pip install sentence_transformers

In [3]:
!pip install llama_index

In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [5]:
documents = SimpleDirectoryReader("/content/Data").load_data()
documents

[Document(id_='10b49be6-07af-4117-860c-dc60c62de085', embedding=None, metadata={'page_label': '1', 'file_name': 'paper (1).pdf', 'file_path': '/content/Data/paper (1).pdf', 'file_type': 'application/pdf', 'file_size': 4549655, 'creation_date': '2024-02-01', 'last_modified_date': '2024-02-01', 'last_accessed_date': '2024-02-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Depth Anything: Unleashing the Power of Large-Scale Unlabeled Data\nLihe Yang1Bingyi Kang2†Zilong Huang2Xiaogang Xu3,4Jiashi Feng2Hengshuang Zhao1†\n1The University of Hong Kong2TikTok3Zhejiang Lab4Zhejiang University\n†corresponding authors\nhttps://depth-anything.github.io\nFigure 1. Our model exhibits impressive generalization ability across extensive unseen scenes. Left two colu

In [6]:
system_prompt = """
You are a QA assistant. Your goal is to answer the questions as accurately as possible based on the instructions and context provided
"""

#supported by llama2
template ="<|USER|>{query_str}<|ASSISTANT|>"
query_wrapper_prompt = SimpleInputPrompt(template = template)

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [8]:
import torch

In [9]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    generate_kwargs={
        "temperature":0.1,
        "do_sample":False
    },
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name = "meta-llama/Llama-2-7b-chat-hf",
    device_map = "auto",

    model_kwargs = {
        "torch_dtype" : torch.float16,
        "load_in_8bit": True
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

In [11]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
)

In [12]:
service_context = ServiceContext.from_defaults(
    chunk_size = 1024,
    llm=llm,
    embed_model = embed_model
)

In [13]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a8078775630>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a8078775630>, id_func=<function default_id_func at 0x7a812f797910>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7a8078e1fd60>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7a8078775630>)

In [14]:
index = VectorStoreIndex.from_documents(documents, service_context = service_context)

In [15]:
query_engine = index.as_query_engine()

In [16]:
resp = query_engine.query("""NotImplementedError                       Traceback (most recent call last)
<ipython-input-31-9e3ca1cb8736> in <cell line: 1>()
----> 1 get_ipython().system('pip install pypdf')
      2 get_ipython().system('pip install -q transformers einops accelerate langchain bitsandbytes datasets')

2 frames
/usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py in _run_command(cmd, clear_streamed_output)
    166     locale_encoding = locale.getpreferredencoding()
    167     if locale_encoding != _ENCODING:
--> 168       raise NotImplementedError(
    169           'A UTF-8 locale is required. Got {}'.format(locale_encoding)
    170       )

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968") \n\n what is this error
""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [17]:
print(resp)



The error message you encountered is a NotImplementedError, which means that the code you are trying to execute is not implemented or available in the current version of Google Colab.

The error message specifically states that a UTF-8 locale is required, but the current locale is not supported. This could be due to a variety of reasons, such as the version of Python you are using or the dependencies installed in your environment.

To resolve this issue, you can try updating your Python version or installing additional dependencies that support UTF-8 encoding. You can also try using a different environment or virtual environment to isolate the issue.

In general, it's important to ensure that your environment is set up correctly and that you have the necessary dependencies installed to run the code you are trying to execute. If you are still encountering issues, you can try searching for more detailed solutions or seeking help from the Google Colab community.


In [18]:
from datasets import load_dataset

dataset = load_dataset("vineetvk/career1000")

Generating train split: 0 examples [00:00, ? examples/s]